In [3]:
import pandas as pd
import numpy as np
import functools as ft
from os import listdir
from os.path import isfile, join
import chardet
import utm
import functools as ft

import sys

sys.path.append("..")
from tools import data_path, output_path, cache_path

### Data origin: 
Data was privately provided by authorities. Not reproducible without original data.

In [4]:
mypath = data_path + "bscv/Abfluss Elbe Saale Oder/Abfluss Elbe Saale Oder/Elbe Oder Saale"
onlyfiles = [mypath +"/" + f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
# # Step 2: Read CSV File in Binary Mode
# with open(onlyfiles[0], 'rb') as f:
#     data = f.read()
 
# # Step 3: Detect Encoding using chardet Library
# encoding_result = chardet.detect(data)
#encoding_result

In [5]:
data_stack = []
meta_stack = []
for x in onlyfiles:
    data_stack.append(pd.read_csv(x, encoding="ISO-8859-9", sep=';', skiprows=13))
    meta_stack.append(pd.read_csv(x, encoding="ISO-8859-9", sep=';', on_bad_lines="skip"))

In [6]:
meta = pd.concat([x[x.columns[1]] for x in meta_stack],axis=1)
meta.index = meta_stack[0].values[:,0]

In [7]:
int(meta_stack[0].iloc[2].values[1])

570910

In [8]:
stack = []
for n,x in enumerate(data_stack):

    a= pd.to_datetime(x["Datum"] + " " + x["Uhrzeit"], format="%d.%m.%Y %H:%M:%S")
    b = x["Durchfluss [m³/s]"]
    c = pd.concat([a,b],axis=1)
    c.columns = ["Datetime", meta_stack[n].iloc[2].values[1] + "_bscv"]
    stack.append(c)

In [9]:
df_final = ft.reduce(lambda left, right: pd.merge(left, right, on='Datetime'), stack)

In [10]:
df_final.index = df_final["Datetime"]
df_final.drop(columns="Datetime", inplace=True)

### Bring meta into format

In [11]:
meta = meta.T.reset_index()

In [12]:
save_index = meta["Stationsnummer:"].values.astype(int)

In [13]:
meta["Lage UTM East:"] = meta["Lage UTM East:"].str.replace(",", ".").astype(float)
meta["Lage UTM North:"] = meta["Lage UTM North:"].str.replace(",", ".").astype(float)

In [14]:
# there is some issue with the coordinates.
#According to wikipedia, the point lies 1km south of hohenstaaten. When keeping the North value the east value should be something like: 442148.512
# Same goes for Eisenhüttenstadt when keeping the value
#https://de.wikipedia.org/wiki/Pegel_Hohensaaten-Finow

In [15]:
meta.loc[6, "Lage UTM East:"] = 442148.512
meta.loc[16, "Lage UTM East:"] = 479828.724

In [16]:
meta.loc[6]["Stationsnummer:"],meta.loc[16]["Stationsnummer:"]

('603080', '603000')

In [17]:
# coordinates
# thuringia parsing
for ind, line in meta.iterrows():
    meta.loc[ind, ["Lage UTM East:", "Lage UTM North:"]] = utm.to_latlon(
        line["Lage UTM East:"], line["Lage UTM North:"], 33, "U"
    )

In [18]:
meta = meta[["Gewässer:","Lage UTM East:",	"Lage UTM North:", "PNP aktuell [m ü. NHN]:"]]
meta.columns = ["R", "X", "Y", "H"]
meta["D"] = np.nan

In [19]:
meta = meta[["R", "X", "Y","D", "H"]]

In [20]:
meta["H"] = (
    meta["H"]
    .astype(str)
    .str.replace(" (DE_DHHN2016_NH)", "", regex=False)
    .str.replace(",", ".")
    .astype(float)
)

In [21]:
meta.index = save_index
meta.index.name = "ID"
bscv = meta

In [22]:
# Another correction
bscv.loc[5930010, "Y"] = 10.933847444656037

In [23]:
bscv["QD"] = -1
bscv["QH"] = 0
bscv["QX"] = 0
bscv["QY"] = 0
bscv["QR"] = 0

In [24]:
bscv.loc[5930010, "QY"] = 1
bscv.loc[603080, "QX"] = 1
bscv.loc[603000, "QX"] = 1

In [25]:
df_final.loc[df_final["570810_bscv"] == "---","570810_bscv"] = None
for x in df_final.columns:
    try:
        df_final[x] = df_final[x].str.replace(",", ".")
    except:
        print(x)

502350_bscv
502210_bscv
502070_bscv
603080_bscv
503050_bscv
5930010_bscv
502180_bscv
502010_bscv


In [26]:
df_final.astype(float).to_csv(output_path + "/bscv_processed.csv")
bscv.to_csv(output_path + "/bscv_meta_data.csv")